Contrast Usage Example

In [2]:
import torch
import torch.nn as nn

from data_utils import *
from train_tools import *

root = '../Data/quickdraw'
dataloaders, dataset_sizes = quickdraw_setter(root=root, batch_size=2048, num_workers=0)
device = 'cuda:0'

KeyboardInterrupt: 

In [2]:
images, v_labels = next(iter(dataloaders['train']))

model = resnet20(dim_in=1, num_classes=20)

with torch.no_grad():
    model.to(device)
    images, v_labels = images.to(device), v_labels.to(device)
    _, v_features = model(images, get_features=True)

ResNet was made


In [3]:
# get fake audio features & audio labels
a_features, a_labels = torch.randn(256, 128).to(device), next(iter(dataloaders['train']))[1][:256].to(device)

In [4]:
# Remind that audio & visual batch size can differ
print(a_features.shape)
print(a_labels.shape)
print(v_features.shape)
print(v_labels.shape)

torch.Size([256, 128])
torch.Size([256])
torch.Size([2048, 128])
torch.Size([2048])


In [5]:
criterion = AVContrastLoss()
loss = criterion(a_features, a_labels, v_features, v_labels)
print(loss)

tensor(4.6381, device='cuda:0')


----
## Save v_features & v_labels to npy file

In [2]:
def tensor_concater(self, tensor1, tensor2):
    if tensor1 is None:
        tensor1 = tensor2
    else:
        tensor1 = torch.cat((tensor1, tensor2), dim=0)

    return tensor1

In [10]:
import torch
import torch.nn as nn

from data_utils import *
from train_tools import *


def tensor_concater(tensor1, tensor2):
    if tensor1 is None:
        tensor1 = tensor2
    else:
        tensor1 = torch.cat((tensor1, tensor2), dim=0)

    return tensor1


root = '../Data/quickdraw'
dataloaders, dataset_sizes = quickdraw_setter(root=root, batch_size=2048, num_workers=10)
device = 'cuda:0'

device = 'cuda:0'
model = resnet20(dim_in=1, num_classes=20).to(device)

images_cat, labels_cat = None, None

with torch.no_grad():
    model.load_state_dict(torch.load('./results/res20_0.8982.pth'))
    for images, v_labels in dataloaders['train']:
        images = images.to(device)
        _, v_features = model(images, get_features=True)
        images_cat = tensor_concater(images_cat, v_features.cpu())
        labels_cat = tensor_concater(labels_cat, v_labels.cpu())
        
print(images_cat.shape)
print(labels_cat.shape)

ResNet was made
torch.Size([1990000, 128])
torch.Size([1990000])


In [13]:
from numpy import save

save('./v_features.npy', images_cat.numpy())
save('./v_labels.npy', labels_cat.numpy())

In [14]:
from numpy import load

v_features_load = load('./v_features.npy')
print(v_features_load.shape)

v_labels_load = load('./v_labels.npy')
print(v_labels_load.shape)

(1990000, 128)
(1990000,)


------